In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pylab as plt
import folium
import requests
from folium.plugins import HeatMap, HeatMapWithTime

In [3]:
df = pd.read_csv("C:/Users/Mainaki Saraf/Desktop/Sem 5/DA/Project/Dataset/Sample_NaStreet_Removed.csv")
df_street=pd.read_csv("street.csv")

In [4]:
#df.iloc[:,25:30].head()
#df.isnull().sum()
df.columns

Index(['Unnamed: 0', 'Unnamed: 0.1', 'Summons Number', 'Plate ID',
       'Registration State', 'Plate Type', 'Issue Date', 'Violation Code',
       'Vehicle Body Type', 'Vehicle Make', 'Issuing Agency', 'Street Code1',
       'Street Code2', 'Street Code3', 'Vehicle Expiration Date',
       'Violation Location', 'Violation Precinct', 'Issuer Precinct',
       'Issuer Code', 'Issuer Command', 'Issuer Squad', 'Violation Time',
       'Time First Observed', 'Violation County',
       'Violation In Front Of Or Opposite', 'House Number', 'Street Name',
       'Intersecting Street', 'Date First Observed', 'Law Section',
       'Sub Division', 'Violation Legal Code', 'Days Parking In Effect    ',
       'From Hours In Effect', 'To Hours In Effect', 'Vehicle Color',
       'Unregistered Vehicle?', 'Vehicle Year', 'Meter Number',
       'Feet From Curb', 'Violation Post Code', 'Violation Description',
       'No Standing or Stopping Violation', 'Hydrant Violation',
       'Double Parking Viola

In [5]:
print(len(df))
invalid = df.loc[(df['Latitude']=='n') & (df['Longitude']=='o')]
print(len(invalid))
df = df[(df['Latitude']!='n') & (df['Longitude']!='o')]
df = df[pd.notnull(df.Hour)]
print(len(df))

1079918
2665
1077238


In [7]:
df_hour=[]
for hour in df.Hour.sort_values().unique():
    df_hour.append(df.loc[df.Hour == hour, ['Latitude', 'Longitude', 'count']].groupby(['Latitude', 'Longitude']).sum().reset_index().values.tolist())

In [8]:
df.Hour.sort_values().unique()

array([ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12.,
       13., 14., 15., 16., 17., 18., 19., 20., 21., 22., 23.])

In [9]:
def generateBaseMap(default_location=[40.693943, -73.985880], default_zoom_start=12):
    base_map = folium.Map(location=default_location, control_scale=True, zoom_start=default_zoom_start)
    return base_map

In [10]:
base_map = generateBaseMap(default_zoom_start=11)
HeatMapWithTime(df_hour, radius=5, gradient={0.2: 'blue', 0.4: 'lime', 0.6: 'orange', 1: 'red'}, min_opacity=0.5, max_opacity=0.8, use_local_extrema=True).add_to(base_map)

In [11]:
base_map.save("hourly map.html")